# Desafio 01

No futebol moderno, empresas especializadas coletam dados e geram estatísticas dos atletas: finalizações, passes, gols, defesas… Esses dados são fundamentais para a
análise de desempenho e ajudam na tomada de decisões de técnicos, preparadores e analistas.

Mas e quando esses dados simplesmente não existem?

Isso é muito comum nas categorias de base, como sub-20, sub-17, sub-15… quanto menor a categoria, maior a escassez de informações.

E é aí que entra o seu desafio como futuro analista de dados!

### Missão

Você foi contratado para criar uma base de dados dos atletas da categoria de base do Fortaleza . Isso mesmo! Você vai estruturar os dados e já deixá-los prontos para futuras análises.

### Requisitos

**Quantidade mínima de atletas: 16.**

Dica: pense como um analista!
Quantos goleiros são necessários?
Quantos zagueiros, laterais, meio-campistas e atacantes?

Monte uma base com **11 titulares e pelo menos 5 reservas** (mas você pode incluir mais, se quiser!).

**Informações obrigatórias de cada atleta:**
    
- Nome
- Idade ou data de nascimento
- Posição
- Nacionalidade
- Número de partidas
- Total de minutos jogados
- Gols marcados
- Assistências

**Bônus:** Se quiser adicionar mais campos (ex: número da camisa, altura, perna dominante…), pode também!

### Validação

**Valide sua estrutura com 3 exemplos:**

Depois de montar sua base de dados, imprima os dados de pelo menos 3 atletas, como nos exemplos abaixo:

- O nome de um **atacante**, com seu número de gols e partidas.
- O nome de um **zagueiro**, com idade, número de partidas e minutos jogados.
- O nome de um **meio-campista**, com posição, nacionalidade e número de assistências.

---

### Web Scraping

A coleta de dados para esse projeto foi realizada através de um web scraping do site Ogol.

Os dados coletados são referentes ao plantel da equipe sub-20 do Fortaleza no Brasileirão sub-20 de 2024. 

In [9]:
import requests
from bs4 import BeautifulSoup

url_base = "https://www.ogol.com.br"
url_fortaleza = "https://www.ogol.com.br/equipe/fortaleza/58110?epoca_id=153"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

response = requests.get(url_fortaleza, headers=headers)

In [10]:
response.status_code

200

Primeiramente, foi realizado a coleta dos links das páginas de cada jogador para poder acessar seus respectivos dados e estatísticas. Esses links estavam atrelados ao nome de cada jogador por meio da tag html `<a>`.   

In [11]:
soup = BeautifulSoup(response.content, 'html.parser')
jogadores_div = soup.select('div.staff div.micrologo_and_text div.text a[href*="/jogador/"]')

jogadores_links = []

for jogador in jogadores_div:
    nome = jogador.get_text(strip=True)
    link = jogador.get('href')
    
    jogadores_links.append({
        "nome": nome,
        "link": url_base + link
    })
    
print(f"Foram encontrados {len(jogadores_links)} jogadores do Fortaleza sub-20 na temporada de 2024.")
print("3 primeiros jogadores encontrados:")

for i, jogador in enumerate(jogadores_links[:3]):
    print(f"{i+1}. {jogador["nome"]} - {jogador["link"]}")

Foram encontrados 58 jogadores do Fortaleza sub-20 na temporada de 2024.
3 primeiros jogadores encontrados:
1. Luizão - https://www.ogol.com.br/jogador/luizao/719536?epoca_id=153
2. Marcelo Catto - https://www.ogol.com.br/jogador/marcelo-catto/974983?epoca_id=153
3. Eduardo Salvador - https://www.ogol.com.br/jogador/eduardo-salvador/991749?epoca_id=153


Após coletar os links de todos os jogadores, foi acessado um por um para extrair tanto os dados pessoais quanto os dados estatísticos referentes ao Brasileirão sub-20 de 2024 de cada jogador. 

A estrutura de dados utilizada para armezanar os dados dos jogadores foi uma **lista de dicionários (estrutura aninhada)**, onde cada elemento da lista é um jogador (dicionário) com suas informações. A decisão de utilizar essa estrutura de dados foi por conta de uma facilidade maior de iterar e buscar informações, além de ser mais simples para exportar para CSV ou JSON. 

In [12]:
jogadores_fortaleza_sub20 = []

print("Iniciando a coleta de dados dos jogadores...\n")

for i, jogador in enumerate(jogadores_links):
    print(f"Coletando dados do jogador {i+1} de {len(jogadores_links)}: {jogador['nome']}")
    
    response = requests.get(jogador["link"], headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    dados_jogador = {}
    
    # Remove o span "Nome", pegando apenas o nome do jogador
    nome = soup.find("div", class_="bio")
    span = nome.find("span").decompose()
    nome = nome.get_text(strip=True)
    
    idade = soup.select_one("div.bio_half span.small").get_text(strip=True)
    idade = idade.replace("(", "").replace(")", "")[0:2] # pega apenas os números da idade
    idade = int(idade)
    
    nacionalidade = soup.select_one("div.bio_half div.micrologo_and_text div.text").get_text(strip=True)
    
    posicao = soup.find("span", string="Posição").parent.get_text(strip=True)
    posicao = posicao.replace("Posição", "").strip().lower()
    
    pe_dominante = soup.find("span", string="Pé preferencial").parent.get_text(strip=True)
    pe_dominante = pe_dominante.replace("Pé preferencial", "").strip()
    
    # Quando não tiver a informação do pé dominante do jogador, atribui "N/A"
    if pe_dominante == "":
        pe_dominante = "N/A"
    
    altura = soup.find("span", string="Altura").parent.get_text(strip=True)
    altura = altura.replace("Altura", "").replace("cm", "").strip() # pega apenas os números da altura
    altura = int(altura)
    
    # Quando não tiver a informação da altura do jogador, atribui None
    if altura == 0:
        altura = None
    
    peso = soup.find("span", string="Peso").parent.get_text(strip=True)
    peso = peso.replace("Peso", "").replace("kg", "").strip() # pega apenas os números do peso
    peso = int(peso)
    
    # Quando não tiver a informação do peso do jogador, atribui None
    if peso == 0:
        peso = None
        
    partidas_jogadas = 0
    minutos_jogados = 0
    gols_marcados = 0
    gols_sofridos = 0
    assistencias = 0
    
    table = soup.find("table", class_="zztable stats")
    
    if not table:
        print(f"O jogador {nome} não disputou o Brasileirão sub-20 na temporada de 2024.\n")
    else:
        for row in table.find_all("tr")[1:-1]:
            campeonato = row.find("td", class_="text")
            
            if campeonato is None:
                continue
            
            campeonato = campeonato.get_text(strip=True)    
            
            if "Brasileiro S20" in campeonato:
                rows = row.find_all("td")[1:]
                
                partidas_jogadas = rows[0].get_text(strip=True)
                partidas_jogadas = int(partidas_jogadas)
                
                minutos_jogados = rows[1].get_text(strip=True)
                minutos_jogados = int(minutos_jogados)
                
                if posicao == "goleiro":
                    gols_sofridos = rows[2].get_text(strip=True)
                    gols_sofridos = int(gols_sofridos)
                else:
                    gols_marcados = rows[2].get_text(strip=True)
                    gols_marcados = int(gols_marcados)
                
                assistencias = rows[3].get_text(strip=True)
                assistencias = int(assistencias)
        
        if partidas_jogadas == 0:
            print(f"O jogador {nome} não disputou o Brasileirão sub-20 na temporada de 2024.\n")      
            
    
    dados_jogador.update({"nome": nome, 
                          "idade": idade,
                          "nacionalidade": nacionalidade,
                          "posicao": posicao,
                          "pe_dominante": pe_dominante,
                          "altura_cm": altura,
                          "peso_kg": peso,
                          "partidas_jogadas": partidas_jogadas,
                          "minutos_jogados": minutos_jogados,
                          "gols_marcados": gols_marcados,
                          "gols_sofridos": gols_sofridos,
                          "assistencias": assistencias})
    
    jogadores_fortaleza_sub20.append(dados_jogador)
    
print("\nColeta dos dados dos jogadores do Fortaleza sub-20 referentes ao Brasileirão sub-20 de 2024, realizada com sucesso.")

Iniciando a coleta de dados dos jogadores...

Coletando dados do jogador 1 de 58: Luizão
Coletando dados do jogador 2 de 58: Marcelo Catto
O jogador Marcelo Chaud Catto não disputou o Brasileirão sub-20 na temporada de 2024.

Coletando dados do jogador 3 de 58: Eduardo Salvador
O jogador Eduardo Salvador Correia não disputou o Brasileirão sub-20 na temporada de 2024.

Coletando dados do jogador 4 de 58: Murilo D'Angelo
Coletando dados do jogador 5 de 58: Carlos Brito
O jogador Carlos Manoel Ferreira Brito não disputou o Brasileirão sub-20 na temporada de 2024.

Coletando dados do jogador 6 de 58: Khendran
O jogador Khendran Andryel Tomaz Santos não disputou o Brasileirão sub-20 na temporada de 2024.

Coletando dados do jogador 7 de 58: Guilherme Moura
Coletando dados do jogador 8 de 58: Cristovam
Coletando dados do jogador 9 de 58: Bruno Moreira
Coletando dados do jogador 10 de 58: Micaell
O jogador Micaell Torres da Silva Antunes não disputou o Brasileirão sub-20 na temporada de 2024.

Depois de coletar todos os dados, vamos salvar esses dados em um arquivo CSV.

In [13]:
import csv

keys = jogadores_fortaleza_sub20[0].keys()

with open("jogadores_fortaleza_sub20.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=keys)
    writer.writeheader()
    writer.writerows(jogadores_fortaleza_sub20)


### Validação da estrutura

In [14]:
print(f"O artilheiro do Fortaleza no Brasileirão sub-20 de 2024 foi o {jogadores_fortaleza_sub20[55]["nome"]} com {jogadores_fortaleza_sub20[55]["gols_marcados"]} gols marcados em {jogadores_fortaleza_sub20[55]["partidas_jogadas"]} partidas.")

O artilheiro do Fortaleza no Brasileirão sub-20 de 2024 foi o Kauê Canela Arroula com 11 gols marcados em 19 partidas.


In [15]:
print(f"O zagueiro {jogadores_fortaleza_sub20[20]["nome"]} de {jogadores_fortaleza_sub20[20]["idade"]} anos, disputou {jogadores_fortaleza_sub20[20]["partidas_jogadas"]} partidas no Brasileirão sub-20 de 2024, totalizando {jogadores_fortaleza_sub20[20]["minutos_jogados"]} minutos de jogo.")

O zagueiro Tony Lucas Graciano da Rocha de 21 anos, disputou 18 partidas no Brasileirão sub-20 de 2024, totalizando 1533 minutos de jogo.


In [16]:
print(f"O {jogadores_fortaleza_sub20[34]["nome"]}, {jogadores_fortaleza_sub20[34]["posicao"]} com nacionalidade do {jogadores_fortaleza_sub20[34]["nacionalidade"]}, deu {jogadores_fortaleza_sub20[34]["assistencias"]} assistência(s) no Brasileirão sub-20 de 2024.")

O Alexsandro Amorim de Freitas Filho, meia (meia ofensivo) com nacionalidade do Brasil, deu 1 assistência(s) no Brasileirão sub-20 de 2024.
